In [1]:
import yfinance as yf
import pandas as pd
import warnings
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
warnings.filterwarnings("ignore")

In [2]:
def get_ticker_data(ticker_name:str, period:str, start_date:str, end_date:str) ->pd.DataFrame:
    """Get ticker data from yahoo finance based on ticker, period, start_date, end_date

    Args:
        ticker (str): stock ticker
        period (str): frequency of data
        start_date (str): start date of data
        end_date (str): end date of data

    Returns:
        pd.DataFrame: resulting dataframe
    """
    ticker = yf.Ticker(ticker_name)
    ticker_data = (
        ticker.history(period = period, start=start_date, end=end_date)
        .reset_index()
        .assign(Ticker_name = ticker_name)
    )
    return ticker_data

In [3]:
def get_ticker_data_list(ticker_list:list, period:str, start_date:str, end_date:str) ->dict:
    """Apply get_ticker_data to a list of tickers

    Args:
        ticker_list (list): list of tickers
        period (str): frequency of data
        start_date (str): start date of data
        end_date (str): end date of data

    Returns:
        dict: A dictionary of ticker dataframes
    """
    storage = {}
    
    for ticker in ticker_list:
        storage[ticker] = get_ticker_data(ticker, period, start_date, end_date)
    
    return storage

In [4]:
tickerSymbols = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL','^GSPC']
MAANG = ['META','AAPL', 'AMZN', 'NFLX', 'GOOGL']

tickerDataFrames = get_ticker_data_list(tickerSymbols, '1d', '2023-1-30', '2024-1-30')

In [5]:
meta_data = tickerDataFrames['META'].filter(items = ['Date', 'Close','Ticker_name'])
google_data = tickerDataFrames['GOOGL'].filter(items = ['Date', 'Close','Ticker_name'])
apple_data = tickerDataFrames['AAPL'].filter(items = ['Date', 'Close','Ticker_name'])
amazon_data = tickerDataFrames['AMZN'].filter(items = ['Date', 'Close','Ticker_name'])
netflix_data = tickerDataFrames['NFLX'].filter(items = ['Date', 'Close','Ticker_name'])
sp500_data = tickerDataFrames['^GSPC'].filter(items = ['Date', 'Close','Ticker_name'])

combined_data_long_format = pd.concat([meta_data, google_data, apple_data, amazon_data, netflix_data, sp500_data], axis=0)

In [6]:
fig = px.line(combined_data_long_format.query("Ticker_name!='^GSPC'"), x='Date', y='Close', color='Ticker_name')
fig.update_layout(
    title={
        'text': 'Stocks Closing Prices',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

fig.show()

In [7]:
for ticker in tickerSymbols:
    tickerDataFrames[ticker]['Return'] = tickerDataFrames[ticker]['Close'].pct_change()

In [8]:
for ticker in MAANG:
    beta, alpha, r_value, p_value, std_err = stats.linregress(tickerDataFrames[ticker]['Return'].iloc[1:], tickerDataFrames['^GSPC']['Return'].iloc[1:])
    tickerDataFrames[ticker]['Beta'] = beta
    tickerDataFrames[ticker]['Alpha'] = alpha
    tickerDataFrames[ticker]['Sharpe_Ratio'] = np.sqrt(252)*tickerDataFrames[ticker]['Return'].mean()/tickerDataFrames[ticker]['Return'].std()

In [9]:
rows_list = []
for ticker in MAANG:
    dict1 = {}
    dict1['Ticker_name'] = ticker
    dict1['Average_Beta'] = tickerDataFrames[ticker]['Beta'].mean()
    dict1['Average_Alpha'] = tickerDataFrames[ticker]['Alpha'].mean()
    dict1['Average_Sharpe_Ratio'] = tickerDataFrames[ticker]['Sharpe_Ratio'].mean()
    rows_list.append(dict1)

average_df = pd.DataFrame(rows_list)

In [10]:
average_df

,Ticker_name,Average_Beta,Average_Alpha,Average_Sharpe_Ratio
0,META,0.188093,0.000038,2.764826
1,AAPL,0.444883,0.000281,1.608109
2,AMZN,0.237761,0.000351,1.659056
3,NFLX,0.139945,0.000536,1.494816
4,GOOGL,0.256274,0.000333,1.726929


In [11]:
print(f"best risk weighted return is {average_df['Average_Sharpe_Ratio'].max()} for {average_df['Ticker_name'].iloc[average_df['Average_Sharpe_Ratio'].argmax()]}")

best risk weighted return is 2.764825806915248 for META


In [12]:
print(f"weighted average beta: {average_df['Average_Beta'].mean()}")

weighted average beta: 0.25339103978267563


In [13]:
(
    pd.concat([tickerDataFrames['META']['Return'],
               tickerDataFrames['GOOGL']['Return'],
               tickerDataFrames['AAPL']['Return'],
               tickerDataFrames['AMZN']['Return'],
               tickerDataFrames['NFLX']['Return']], axis=1)
    .dropna()
    .set_axis(['META', 'GOOGL', 'AAPL', 'AMZN', 'NFLX'], axis=1)
    .corr()
)

,META,GOOGL,AAPL,AMZN,NFLX
META,1.000000,0.609519,0.533567,0.594361,0.314620
GOOGL,0.609519,1.000000,0.514389,0.595888,0.319017
AAPL,0.533567,0.514389,1.000000,0.423092,0.390472
AMZN,0.594361,0.595888,0.423092,1.000000,0.359034
NFLX,0.314620,0.319017,0.390472,0.359034,1.000000
